In [12]:
import cv2
import paho.mqtt.client as mqtt
import numpy as np
import time

# MQTT 설정
#broker_address = "192.168.14.128"
broker_address ="70.12.225.174"
port = 1883
topic = "AGV/camera"

# MQTT 클라이언트 초기화 및 연결
client = mqtt.Client()
client.connect(broker_address, port)

0

In [13]:
gst_str = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=(int)640, height=(int)480, "
    "format=(string)NV12, framerate=(fraction)30/1 ! nvvidconv ! video/x-raw, "
    "format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink"
)

In [14]:
# 카메라 초기화
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(gst_str, cv2.CAP_GSTREAMER)

In [15]:
def send_frame():
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        #resized_frame = cv2.resize(frame, (640, 480))
        # 이미지를 JPEG로 인코딩
        _, buffer = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
        
        with open("testimg.jpg", "wb")as f:
            f.write(buffer.tobytes())
        # MQTT로 이미지 전송
        jpg_as_text = buffer.tobytes()
        client.publish(topic, jpg_as_text)
        #time.sleep(0.1)  # 전송 간격 (초 단위)
        #print("pub")
        #print(len(buffer))

In [ ]:
try:
    send_frame()
except KeyboardInterrupt:
    cap.release()
    client.disconnect()

In [20]:
send_frame()